### 인터파크 베스트셀러

In [104]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [98]:
site = 'http://book.interpark.com'
url='http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
lis = soup.select('.rankBestContentList > ol > li')
len(lis), type(lis)

(15, bs4.element.ResultSet)

In [84]:
li = lis[0]
href = li.select_one('.coverImage').find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354595636&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [85]:
title = li.select_one('.itemName').getText().strip()
title

'저주토끼'

In [86]:
author = li.select_one('.author').getText().strip()
author

'정보라(정도경)'

In [87]:
company = li.select_one('.company').getText().strip()
company

'아작'

In [88]:
price = li.select_one('.price > em').get_text().replace(',','')
price = int(price)
price

13320

In [89]:
#rank = li.select_one('.rankBtn_ctrl').get_attribute_list('class')[1][-1]
rank = li.select('.rankBtn_ctrl')
rank_n = ''
for r in rank:
    rank_n += r.get_attribute_list('class')[1][-1]
rank_n = int(rank_n)
rank_n

1

In [99]:
url = site + href
#print(url)
res = requests.get(url)
sub_soup = BeautifulSoup(res.text, 'html.parser')

http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354595636&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001


In [101]:
sub_lis = sub_soup.select('.bInfo_txt > li')
len(sub_lis)

5

In [102]:
for li in sub_lis:
    print(li.getText())

저 : 정보라
출판사 : 아작
발행 : 2022년 04월 01일
쪽수 : 328
ISBN : 9791166686665


In [136]:
for li in sub_lis:
    text = li.getText().strip()
    print(text)
    trans = '-'
    if '발행 :' in text:
        data = text.split(':')[1].strip()
        data = re.sub('[^0-9]', '', data)
        # print('date',data)
    elif '쪽수 :' in text:
        page = text.split(':')[1].strip()
        print('쪽수',page)
print(data, page, trans)#trans가 없는 경우가 있음

저 : 최태성
출판사 : 이투스북
발행 : 2021년 12월 21일
쪽수 : 392
쪽수 392
ISBN : 9791138902823
20211221 392 -


- 모든 코드를 조합

In [137]:
site = 'http://book.interpark.com'
url='http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
lis = soup.select('.rankBestContentList > ol > li')
lines = []

for li in lis:
    href = li.select_one('.coverImage').find('a')['href']
    title = li.select_one('.itemName').getText().strip()
    #author = li.select_one('.author').getText().strip()
    company = li.select_one('.company').getText().strip()
    price = int(li.select_one('.price > em').get_text().replace(',',''))
    rank = li.select('.rankBtn_ctrl')
    rank_n = ''
    for r in rank:
        rank_n += r.get_attribute_list('class')[1][-1]
    rank_n = int(rank_n)
    
    url = site + href
    res = requests.get(url)
    sub_soup = BeautifulSoup(res.text, 'html.parser')
    sub_lis = sub_soup.select('.bInfo_txt > li')
    for li in sub_lis:
        text = li.getText().strip()
        if '발행 :' in text:
            date = text.split(':')[1].strip()
            date = re.sub('[^0-9]', '', data)
        elif '쪽수 :' in text:
            page = text.split(':')[1].strip()
        elif '저 :' in text:
            author = text.split(':')[1].strip()
    lines.append([rank_n, title, author, company, page, date, price])

df=pd.DataFrame(lines, columns=['rank', 'title', 'author', 'company', 'page', 'date', 'price'])
df
    

,rank,title,author,company,page,date,price
0,1,저주토끼,정보라,아작,328,20220401,13320
1,2,긴긴밤,루리,문학동네,144,20220401,10350
2,3,럭키(10만부 기념 황금열쇠 양장 특별판),김도윤,북로망스,280,20220401,14400
3,4,꽃샘바람에 흔들린다면 너는 꽃,류시화,수오서재,160,20220401,12150
4,4,파이어 FIRE,강환국,페이지2북스,428,20220401,16200
5,4,웰씽킹(10만 부 기념 한정판 골드 에디션),켈리 최,다산북스,316,20220401,14400
6,4,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,나무옆의자,268,20220401,12600
7,8,김학렬의 부동산 투자 절대 원칙,김학렬(빠숑),에프엔미디어,540,20220401,22500
8,9,파이어족 강환국의 하면 된다! 퀀트 투자,강환국,에프엔미디어,392,20220401,16830
9,10,나의 투자는 새벽 4시에 시작된다,유목민,리더스북,386,20220401,17820


 정보라
출판사 
결과 :  20220401 392 최태성
